In [91]:
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json
import os


def get_service(credential_json='credentials.json'):
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credential_json, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)


def get_messages(service, pageToken=None):
    if pageToken:
        return service.users() \
            .messages() \
            .list(userId='me', maxResults=500, pageToken=pageToken) \
            .execute()
    return service.users() \
        .messages() \
        .list(userId='me', maxResults=500) \
        .execute()


def get_all_messages(service):
    totalResults = list()
    pageToken = None
    while True:
        results = get_messages(service, pageToken=pageToken)
        totalResults.append(results)
        if 'nextPageToken' in results:
            pageToken = results['nextPageToken']
        else:
            break
    return totalResults


def count_messages(all_messages):
    totalMessages = 0
    for result in totalResults:
        totalMessages += len(result.get('messages'))
    return totalMessages

async def get_message_content(service, msgId):
    return service.users().messages().get(userId='me',id=msgId,format='metadata').execute()


async def header_as_json(header):
    fields = 'Date,From,To'.split(',')
    mail_header = {}
    for item in header:
        name = item.get('name')
        if name in fields:
            mail_header[name] = item.get('value')
    return mail_header


async def get_headers(service, msgId):
    message = await get_message_content(service, msgId)
    try:
        return await header_as_json(message['payload']['headers'])
    except:
        return {'Date': None,'From': None,'To': None}

In [92]:
#service = get_service(credential_json='credentials.json')
#all_messages = get_all_messages(service)
#total_messages = count_messages(all_messages)

#ids = list()
#for messages in all_messages:
#    ids = ids + messages.get('messages')

headers = list()
for msg in ids[:100]:
    header = await get_headers(service, msg.get('id'))
    headers.append(header)

with open('result_headers', 'w') as fopen:
    for header in headers:
        fopen.write(json.dumps(header) + '\n')

TypeError: object dict can't be used in 'await' expression